In [3]:
from dotenv import load_dotenv
load_dotenv()
import os
from openai import OpenAI
from pinecone import Pinecone, ServerlessSpec


In [6]:
# Initialize Pinecone
pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))

# Create a Pinecone index
pc.create_index(
    name="rag",
    dimension=1536,
    metric="cosine",
    spec=ServerlessSpec(cloud="aws", region="us-east-1"),
)

PineconeApiException: (409)
Reason: Conflict
HTTP response headers: HTTPHeaderDict({'content-type': 'text/plain; charset=utf-8', 'access-control-allow-origin': '*', 'vary': 'origin,access-control-request-method,access-control-request-headers', 'access-control-expose-headers': '*', 'x-pinecone-api-version': '2024-07', 'X-Cloud-Trace-Context': '83b5f5e9bddc4172c644c3cc5e36e0c7', 'Date': 'Wed, 21 Aug 2024 23:28:07 GMT', 'Server': 'Google Frontend', 'Content-Length': '85', 'Via': '1.1 google', 'Alt-Svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'})
HTTP response body: {"error":{"code":"ALREADY_EXISTS","message":"Resource  already exists"},"status":409}


In [7]:
import json
# Load the review data
data = json.load(open("reviews.json"))
data['reviews']

[{'professor': 'Dr. Alice Johnson',
  'review': 'Great professor! Explains concepts clearly and is very approachable.',
  'subject': 'Calculus',
  'stars': 5},
 {'professor': 'Prof. Bob Smith',
  'review': 'The lectures were engaging, but the exams were tough.',
  'subject': 'Physics',
  'stars': 4},
 {'professor': 'Dr. Carol White',
  'review': 'Very knowledgeable but tends to go off-topic often.',
  'subject': 'History',
  'stars': 3},
 {'professor': 'Prof. David Lee',
  'review': 'Amazing class! The professor makes learning fun.',
  'subject': 'Computer Science',
  'stars': 5},
 {'professor': 'Dr. Emily Clark',
  'review': 'The course material was difficult, but the professor was very supportive.',
  'subject': 'Chemistry',
  'stars': 4},
 {'professor': 'Prof. Frank Harris',
  'review': 'The professor was too strict and the assignments were overwhelming.',
  'subject': 'Economics',
  'stars': 2},
 {'professor': 'Dr. Grace Miller',
  'review': "One of the best professors I've ever ha

In [8]:
processed_data = []
client = OpenAI()

# Create embeddings for each review
for review in data["reviews"]:
    response = client.embeddings.create(
        input=review['review'], model="text-embedding-3-small"
    )
    embedding = response.data[0].embedding
    processed_data.append(
        {
            "values": embedding,
            "id": review["professor"],
            "metadata":{
                "review": review["review"],
                "subject": review["subject"],
                "stars": review["stars"],
            }
        }
    )

In [9]:
processed_data[0]

{'values': [-0.0419491,
  -0.036433302,
  -0.047997124,
  0.022510748,
  -0.015603905,
  -0.031643268,
  -0.026127467,
  0.041441068,
  -0.024845287,
  -0.044707,
  0.018422283,
  -0.017527176,
  -0.00036609452,
  -0.0053192377,
  0.0046902434,
  -0.012991157,
  -0.008672868,
  0.007263678,
  0.027143536,
  0.044900537,
  0.013245175,
  -0.014321723,
  0.006093385,
  0.049497034,
  -0.05220655,
  -0.030433662,
  0.014007226,
  0.018760974,
  -0.010565899,
  0.0059391605,
  0.08254345,
  -0.006731452,
  -0.024591269,
  -0.0578675,
  -0.034473743,
  0.037957404,
  -0.018228747,
  0.021930138,
  -0.017757,
  0.0154708475,
  0.012337971,
  0.03261095,
  -0.009840136,
  -0.0064713866,
  0.019269008,
  -0.0020623766,
  -0.04366674,
  -0.0040521766,
  0.013160503,
  0.039336354,
  -0.030094972,
  -0.019510929,
  0.041416876,
  0.021143895,
  -0.06280269,
  0.032320645,
  0.022849439,
  0.0047597955,
  0.0014296019,
  -0.032078724,
  0.04944865,
  0.007995489,
  -0.0004354578,
  -0.011588016,


In [10]:
# Insert the embeddings into the Pinecone index
index = pc.Index("rag")
upsert_response = index.upsert(
    vectors=processed_data,
    namespace="ns1",
)

In [11]:
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'ns1': {'vector_count': 20}},
 'total_vector_count': 20}